In [1]:
# hide
# default_exp init

# Imports

In [2]:
# export

from pathlib import Path
import sys
import shutil

from fastcore.script import Param, call_parse

In [3]:
%load_ext autoreload
%autoreload 2

# Initialise a new Sciflow Project

In [4]:
# export 

env_file_blank = """
# export USER=
# export ODBC_DRIVER=
# export ODBC_USER=
# export ODBC_PWD=
# export ODBC_PORT=
# export ODBC_HOST=
# export SSL_CERTS=
# export SCIFLOW_BUCKET=
# export SCIFLOW_BUCKET_TABLE_ALIAS=

"""

In [5]:
# export

def write_env_file(sciflow_dir: Path = None):
    if sciflow_dir is None:
        sciflow_dir = Path('~/.sciflow').expanduser()
    env_path = Path(sciflow_dir, 'env').resolve()
    if not env_path.exists():
        if not sciflow_dir.exists():
            os.mkdir(sciflow_dir)
        with open(env_path, 'w') as env_file:
            env_file.write(env_file_blank)
        print(f'Wrote new SciFlow environment file to: {env_path}')
    else:
        print(f'Skipping SciFlow environment file creation - already exists')

In [6]:
sciflow_dir = Path('test/.sciflow')
env_path = os.path.join(sciflow_dir, 'env')

write_env_file(sciflow_dir)

with open(env_path, 'r') as env_file:
        env_file_contents = env_file.read()
assert(env_file_contents == env_file_blank)

write_env_file()

with open(env_path, 'r') as env_file:
        env_file_contents = env_file.read()
assert(env_file_contents == env_file_blank)

with open(env_path, 'w') as env_file:
    env_file.write("test")
write_env_file(sciflow_dir)
with open(env_path, 'r') as env_file:
        env_file_contents = env_file.read()
assert("test" == env_file_contents)
        
Path('test/.sciflow/env').unlink()
assert(not Path('test/.sciflow/env').exists())

Wrote new SciFlow environment file to: /home/sagemaker-user/git/sciflow/nbs/test/.sciflow/env
Skipping SciFlow environment file creation - already exists
Skipping SciFlow environment file creation - already exists


In [7]:
# export


@call_parse
def sciflow_init(project_root: Param("The root directory of the project", Path) = None):
    if project_root is None:
        project_root = Path('.').resolve()
    # Add project_root to PYTHONPATH
    sys.path.append(str(project_root.resolve()))
    
    # TODO - Get latest templates files from web - if has internet connection
    
    # Create sciflow env file if it doesn't exist
    write_env_file()

In [8]:
sciflow_init(Path('../examples'))

Skipping SciFlow environment file creation - already exists


In [9]:
assert any([p for p in sys.path if p == str(Path('../examples').resolve())])

In [10]:
from sciflow_examples.abalone import preprocess, train